In [1]:
import pandas as pd

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [26]:
# {start} chamar dados
# data = pd.read_csv('/home/wilmer/Documentos/Codes/WORKSHOP/data/historicalData.csv')
# data_rote = pd.read_csv('/home/wilmer/Documentos/Codes/WORKSHOP/data/TrainRoutes.csv')
# {end} chamar dados

data = pd.read_csv('/home/wilmer/Documentos/Codes/WORKSHOP/data/testInstance_Amtrak.csv')

In [27]:
data['DepartureDate'] = data['DepartureDateTime'].str[:10]
data['DepartureTime'] = data['DepartureDateTime'].str[10:]

# data_rote = data_rote.groupby(by=['Service','Num'])['Route'].unique()
# data_rote = data_rote.reset_index()

In [28]:
data.head()

,Unnamed: 0,Market,TrainNumber,DepartureDateTime,ArrivalDateTime,DBD,Origin,Destination,CommercialClass,Revenue,Bookings,Capacity,NbLegs,DepartureDate,DepartureTime
0,0,NYP-BAL,185,2023-09-15 09:35:00+00:00,2023-09-15 12:21:00+00:00,0,NYP,BAL,Y,185.000000,1,493,7,2023-09-15,09:35:00+00:00
1,1,NYP-BAL,185,2023-09-15 09:35:00+00:00,2023-09-15 12:21:00+00:00,1,NYP,BAL,YA,164.666667,3,493,7,2023-09-15,09:35:00+00:00
2,2,NYP-BAL,185,2023-09-15 09:35:00+00:00,2023-09-15 12:21:00+00:00,1,NYP,BAL,YC,124.000000,1,493,7,2023-09-15,09:35:00+00:00
3,3,NYP-BAL,185,2023-09-15 09:35:00+00:00,2023-09-15 12:21:00+00:00,2,NYP,BAL,YA,165.000000,1,493,7,2023-09-15,09:35:00+00:00
4,4,NYP-BAL,185,2023-09-15 09:35:00+00:00,2023-09-15 12:21:00+00:00,4,NYP,BAL,JD,159.000000,1,493,7,2023-09-15,09:35:00+00:00


In [29]:
# {START} ANONIMIZAR OS DADOS
# --data da instancia
replace_train_number = {val: indx + 1  for indx,val in enumerate(sorted(data['TrainNumber'].unique()))}
data['TrainNumber'] = data['TrainNumber'].map(replace_train_number, na_action='ignore')

replace_comer_class = {'Y': 'Z-1','YA': 'Z-2','YB': 'Z-3','YC': 'Z-4','YD': 'Z-5','YE': 'Z-6','YF': 'Z-7','YG': 'Z-8','YH': 'Z-9','JY': 'P-1','JA': 'P-2','JB': 'P-3','JC': 'P-4','JD': 'P-5','JE': 'P-6','JF': 'P-7','JG': 'P-8','JH': 'P-9'}
data['CommercialClass'] = data['CommercialClass'].map(replace_comer_class, na_action='ignore')

origin = data['Origin'].unique().tolist()
destin = data['Destination'].unique().tolist()
stations = sorted(destin + [i for i in origin if i not in destin])

# replace_stations = {val:'S'+str(pos+1) for pos, val in enumerate(stations)}
# data['Origin'] = data['Origin'].map(replace_stations, na_action='ignore')
# data['Destination'] = data['Destination'].map(replace_stations, na_action='ignore')

# data['Market'] = data['Origin'].astype('str') + '-' + data['Destination'].astype('str')

data[['Vagon','Class']] = data['CommercialClass'].str.split('-', expand=True)

# --data das rotas
# def reem_station(fila, replace_stations):#
#     fila = fila[0]
#     fila = fila.split('-')
#     fila = [replace_stations[i] for i in fila]
#     return fila

# data_rote['Route'] = data_rote['Route'].apply(reem_station, replace_stations=replace_stations)
# data_rote['Num'] = data_rote['Num'].map(replace_train_number, na_action='ignore')

# {END} ANONIMIZAR OS DADOS

In [ ]:
# data[(data['TrainNumber']==71) & (data['DepartureDate']=='2023-10-26')]['Capacity'].unique() #& (data['Market']=='S8-S5')

In [30]:
# {START} ORGANIZAR INSTANCIA
instance_dem = data.groupby(by=['Origin','Destination','Vagon', 'Class', 'DBD', 'TrainNumber', 'DepartureDate'])['Bookings'].sum()
instance_dem = instance_dem.reset_index()

instance_preco = data.groupby(by=['Origin','Destination','Vagon', 'Class', 'TrainNumber', 'DepartureDate'])['Revenue'].max()
instance_preco = instance_preco.reset_index()
# {END} ORGANIZAR INSTANCIA

In [31]:
int_perco = instance_preco[['Origin', 'Destination', 'Vagon', 'Class', 'Revenue']]
int_dem = instance_dem[['Origin', 'Destination', 'Vagon', 'Class', 'DBD', 'Bookings']]
int_perco.to_csv('/home/wilmer/Documentos/Codes/tesis/instancias/preco.csv', index=False)
int_dem.to_csv('/home/wilmer/Documentos/Codes/tesis/instancias/demanda.csv', index=False)

In [ ]:
num_trem = 71
data_depature = '2023-10-26'

demanda = instance_dem[(instance_dem['TrainNumber']==num_trem) & (instance_dem['DepartureDate']==data_depature)]
demanda.drop(columns=['TrainNumber','DepartureDate'], inplace=True)

preco = instance_preco[(instance_preco['TrainNumber']==num_trem) & (instance_preco['DepartureDate']==data_depature)]
preco.drop(columns=['TrainNumber','DepartureDate'], inplace=True)
# preco = preco.sort_values(by=['Origin', 'Destination', 'Vagon', 'Revenue'], ascending=[True, True, True, False])

rota = data_rote[data_rote['Num']==num_trem]

In [ ]:
demanda.to_csv('/home/wilmer/Documentos/Codes/tesis/instancias/instancia3/demanda.csv', index=False)
preco.to_csv('/home/wilmer/Documentos/Codes/tesis/instancias/instancia3/preco.csv', index=False)
rota.to_csv('/home/wilmer/Documentos/Codes/tesis/instancias/instancia3/rota.csv', index=False)

In [ ]:
# {START} OBTER AS ROTAS COM MAIS TRECHOS
instance = data.groupby(by=['Market','Vagon', 'Class', 'DBD', 'TrainNumber', 'DepartureDate'])['Bookings'].sum()
instance = instance.reset_index()
long_rote = instance.groupby(by=['TrainNumber','DepartureDate'])['Market'].nunique()
long_rote = long_rote.reset_index()
long_rote = long_rote.sort_values('Market', ascending=False)
long_rote.head(30)
# {END} OBTER AS ROTAS COM MAIS TRECHOS



# # {START} GRAFICAR UMA ROTA
# import networkx as nx
# import matplotlib.pyplot as plt

# # para obter a rota com mais trechos
# filtro = instance[(instance['TrainNumber']==68) & (instance['DepartureDate']=='2023-11-21')]['Market'].unique()


# lista_tuplas  = [(str(i).split('-')[0], str(i).split('-')[1])  for i in filtro]

# # Criar um grafo dirigido
# grafo = nx.DiGraph()

# # Adicionar bordas ao grafo desde as tuplas
# grafo.add_edges_from(lista_tuplas)

# # Desenhar o grafo
# plt.figure(figsize=(25, 13))
# fig = nx.draw(grafo, with_labels=True, node_color='lightblue', node_size=800, font_size=12, font_weight='bold')

# # apresentar o grafico
# plt.show()

# # {END} GRAFICAR UMA ROTA